# Connect to the Database

In [ ]:
# Import the necessary packages
import pandas as pd
# import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt
from sqlalchemy import create_engine, MetaData, inspect, select, Table, column, insert, delete, update
# from datetime import date, datetime
# from decimal import Decimal
# import json
# from bson import Decimal128

# import psycopg2

In [ ]:
# This package allows us to prompt for the database password without it being shown.
from getpass import getpass

In [ ]:
from sqlalchemy.engine import URL

pw = getpass("Enter the PostgreSQL password: ")

url = URL.create(
    drivername="postgresql+psycopg2",
    username="postgres",
    password=pw,
    host="localhost",
    port=5432,
    database="CS_MedDispApp"
)

engine = create_engine(url)

Enter the PostgreSQL password: ········


In [ ]:
# Now run a command to list out all the tables in this Database
inspector = inspect(engine)

# The inspectpr object has several useful items, but we want to get the table names - use get_table_names()
table_names = inspector.get_table_names()

# Now print out the list of table names
print(table_names)

['ingredient', 'contain', 'drug', 'patient', 'drugintakeevent', 'nurse', 'includes', 'nottotakewith']


In [ ]:
# Define metadata
metadata = MetaData()

# Define tables
ingredient = Table("ingredient", metadata, autoload_with=engine)
contain = Table("contain", metadata, autoload_with=engine)
drug = Table("drug", metadata, autoload_with=engine)
patient = Table("patient", metadata, autoload_with=engine)
drugintakeevent = Table("drugintakeevent", metadata, autoload_with=engine)
nurse = Table("nurse", metadata, autoload_with=engine)
includes = Table("includes", metadata, autoload_with=engine)
nottotakewith = Table("nottotakewith", metadata, autoload_with=engine)

# Inserting

#### Inserting new patients

In [ ]:
def insertNewPatient():
    data = input("List the values of the following fields in the same order provided, where each value is separated by \
a \"|\":\nPatientID|PatientName|PatientGender|PatientDOB\n")

    values = data.split("|")

    stmt = insert(patient).values(patientid=values[0], patientname=values[1], patientgender=values[2],
                                  patientdob=values[3])

    with engine.begin() as conn:
        conn.execute(stmt)

    print("Inserted: ", values)

In [ ]:
insertNewPatient()

List the values of the following fields in the same order provided, where each value is separated by a "|":
PatientID|PatientName|PatientGender|PatientDOB
0856139|Stella Miliatis|Female|05/16/2004
Inserted:  ['0856139', 'Stella Miliatis', 'Female', '05/16/2004']


In [ ]:
# Query to check that it worked
stmt = "select patientid, patientname, patientgender, patientdob " \
       "from patient " \
       "where patientid = '0856139'"

results = pd.read_sql(stmt, engine)
results.head()

,patientid,patientname,patientgender,patientdob
0,0856139,Stella Miliatis,Female,2004-05-16


#### Insert ingredients included in new drugs

In [ ]:
def insertNewIngredient():
    data = input("List the values of the following fields in the same order provided, where each value is separated by a \"|\":\nIngredientID|IngredientName\n")
    values = data.split("|")
    stmt = insert(ingredient).values(ingredientid=values[0], ingredientname=values[1])

    with engine.begin() as conn:
        conn.execute(stmt)

    print("Inserted: ", values)

In [ ]:
insertNewIngredient()

List the values of the following fields in the same order provided, where each value is separated by a "|":
IngredientID|IngredientName
I-203608|H2O
Inserted:  ['I-203608', 'H2O']


In [ ]:
# Query to check that it worked
stmt = "select ingredientid, ingredientname " \
       "from ingredient " \
       "where ingredientid = 'I-203608'"

results = pd.read_sql(stmt, engine)
results.head()

,ingredientid,ingredientname
0,I-203608,H2O


#### New nurse/medical staff

In [ ]:
def insertNewNurse():
    data = input("List the values of the following fields in the same order provided, where each value is separated by a \"|\":\nNurseID|NurseName\n")
    values = data.split("|")
    stmt = insert(nurse).values(nurseid=values[0], nursename=values[1])

    with engine.begin() as conn:
        conn.execute(stmt)

    print("Inserted: ", values)

In [ ]:
insertNewNurse()

List the values of the following fields in the same order provided, where each value is separated by a "|":
NurseID|NurseName
N-654322|Sheng Tan
Inserted:  ['N-654322', 'Sheng Tan']


In [ ]:
# Query to check that it worked
stmt = "select nurseid, nursename " \
       "from nurse " \
       "where nurseid = 'N-654322'"

results = pd.read_sql(stmt, engine)
results.head()

,nurseid,nursename
0,N-654322,Sheng Tan


# Updating

#### Updating patients’ and nurses’ personal information, such as any changes to their name

In [ ]:
def updatePatient():
    patient_id = input("Type the PatientID\n")
    patient_field_str = input("Type the desired field to update (options: PatientID, PatientName, PatientGender, PatientDOB)\n")
    patient_newvalue = input("Type the new value\n")

    if patient_field_str == 'PatientID':
        patient_field = 'patientid'
    elif patient_field_str == 'PatientName':
        patient_field = 'patientname'
    elif patient_field_str == 'PatientGender':
        patient_field = 'patientgender'
    else:
        patient_field = 'patientdob'

    update_stmt = (
        update(patient)
        .where(patient.c.patientid == patient_id)
        .values({patient_field: patient_newvalue})
    )

    with engine.begin() as conn:
        conn.execute(update_stmt)

    print("New value: ", patient_newvalue)

In [ ]:
updatePatient()

Type the PatientID
0856139
Type the desired field to update (options: PatientID, PatientName, PatientGender, PatientDOB)
PatientName
Type the new value
Stella J Miliatis
New value:  Stella J Miliatis


In [ ]:
# Query to check that it worked
stmt = "select patientid, patientname, patientgender, patientdob " \
       "from patient " \
       "where patientid = '0856139'"

results = pd.read_sql(stmt, engine)
results.head()

,patientid,patientname,patientgender,patientdob
0,0856139,Stella J Miliatis,Female,2004-05-16


# Deleting

#### Deleting patients in case they switch doctors, they become deceased, move cities. etc.

In [ ]:
def deletePatient():
    data = input("List the PatientID\n")

    stmt = delete(patient).where(patient.c.patientid == data)

    with engine.begin() as conn:
        conn.execute(stmt)

    print("Deleted PatientID: ", data)

In [ ]:
deletePatient()

List the PatientID
0856139
Deleted PatientID:  0856139


In [ ]:
# Query to check that it worked
stmt = "select patientid, patientname, patientgender, patientdob " \
       "from patient " \
       "where patientid = '0856139'"

results = pd.read_sql(stmt, engine)
results.head()

,patientid,patientname,patientgender,patientdob


#### Deleting ingredients

In [ ]:
def deleteIngredient():
    data = input("List the IngredientID\n")

    stmt = delete(ingredient).where(ingredient.c.ingredientid == data)

    with engine.begin() as conn:
        conn.execute(stmt)

    print("Deleted IngredientID: ", data)

In [ ]:
deleteIngredient()

List the IngredientID
I-203608
Deleted IngredientID:  I-203608


In [ ]:
# Query to check that it worked
stmt = "select ingredientid, ingredientname " \
       "from ingredient " \
       "where ingredientid = 'I-203608'"

results = pd.read_sql(stmt, engine)
results.head()

,ingredientid,ingredientname


#### Deleting nurses in case they no longer work at the hospital

In [ ]:
def deleteNurse():
    data = input("List the NurseID\n")

    stmt = delete(nurse).where(nurse.c.nurseid == data)

    with engine.begin() as conn:
        conn.execute(stmt)

    print("Deleted NurseID: ", data)

In [ ]:
deleteNurse()

List the NurseID
N-654322
Deleted NurseID:  N-654322


In [ ]:
# Query to check that it worked
stmt = "select nurseid, nursename " \
       "from nurse " \
       "where nurseid = 'N-654322'"

results = pd.read_sql(stmt, engine)
results.head()

,nurseid,nursename


# Querying

#### Querying patients’ medical histories (their past drug intake events/previous drugs they’ve taken)

In [ ]:
def get_patient_med_history():

    patient_id = input("List the PatientID\n")

    stmt = (
        select(
            drugintakeevent.c.dienumber,
            drugintakeevent.c.diedate,
            drugintakeevent.c.dietime,
            includes.c.drugid,
            drug.c.drugname,
            includes.c.quantity,
        )
        .select_from(
            drugintakeevent
            .join(includes, drugintakeevent.c.dienumber == includes.c.dienumber)
            .join(drug, includes.c.drugid == drug.c.drugid)
        )
        .where(drugintakeevent.c.patientid == patient_id)
        .order_by(drugintakeevent.c.diedate, drugintakeevent.c.dietime, drug.c.drugname)
    )

    return pd.read_sql(stmt, engine)

In [ ]:
get_patient_med_history()

List the PatientID
P-123461


,dienumber,diedate,dietime,drugid,drugname,quantity
0,DIE-246818,2021-01-10,11:50:09,D-987601,Paracetamol,2


#### Querying drugs to view their ingredients as well as other drugs they cannot be taken with

In [ ]:
def get_drug_details():

    drug_id = input("List the DrugID\n")

    # 1) Ingredients for this drug
    ingredients_stmt = (
        select(
            ingredient.c.ingredientid,
            ingredient.c.ingredientname,
            contain.c.amount,
        )
        .select_from(
            contain.join(
                ingredient,
                contain.c.ingredientid == ingredient.c.ingredientid
            )
        )
        .where(contain.c.drugid == drug_id)
        .order_by(ingredient.c.ingredientname)
    )

    # 2) Drugs that cannot be taken with this one
    # Here we interpret "cannot be taken with" as rows where this drug is DrugIDTaking.
    interactions_stmt = (
        select(
            drug.c.drugid,
            drug.c.drugname,
        )
        .select_from(
            nottotakewith.join(
                drug,
                nottotakewith.c.drugidnottotakewith == drug.c.drugid
            )
        )
        .where(nottotakewith.c.drugidtaking == drug_id)
        .order_by(drug.c.drugname)
    )

    with engine.connect() as conn:
        ingredients = conn.execute(ingredients_stmt).fetchall()
        interactions = conn.execute(interactions_stmt).fetchall()

    ingredients_df = pd.read_sql(ingredients_stmt, engine)
    interactions_df = pd.read_sql(interactions_stmt, engine)

    return pd.read_sql(ingredients_stmt, engine), pd.read_sql(interactions_stmt, engine)

In [ ]:
get_drug_details()

List the DrugID
D-987602


(  ingredientid              ingredientname  amount
 0     I-203574           Calcium Carbonate      15
 1     I-203575              Candelilla Wax       1
 2     I-203562       Croscarmellose Sodium       8
 3     I-203576     Hydroxypropyl Cellulose       5
 4     I-203573         Lactose Monohydrate      40
 5     I-203565          Magnesium Stearate       3
 6     I-203564  Microcrystalline Cellulose      40
 7     I-203568              Polysorbate 80       1,
       drugid         drugname
 0   D-987654        Adagrasib
 1   D-987652       Amprenavir
 2   D-987653       Atazanavir
 3   D-987651       Boceprevir
 4   D-987650   Clarithromycin
 5   D-987649       Clofibrate
 6   D-987648       Colchicine
 7   D-987647       Conivaptan
 8   D-987646     Cyclosporine
 9   D-987645          Danazol
 10  D-987644        Darunavir
 11  D-987643     Erythromycin
 12  D-987642  Fenofibric Acid
 13  D-987641      Fluconazole
 14  D-987640        Indinavir
 15  D-987639     Ketoconazole
 16 

#### Viewing which nurses administered certain drug intake events

In [ ]:
def get_nurse_drugintakeevents():

    nurse_id = input("List the NurseID\n")

    stmt = (
        select(
            drugintakeevent.c.dienumber,
            drugintakeevent.c.diedate,
            drugintakeevent.c.dietime,
            drugintakeevent.c.patientid,
        )
        .where(drugintakeevent.c.nurseid == nurse_id)
        .order_by(drugintakeevent.c.diedate, drugintakeevent.c.dietime)
    )

    return pd.read_sql(stmt, engine)

In [ ]:
get_nurse_drugintakeevents()

List the NurseID
N-653928


,dienumber,diedate,dietime,patientid
0,DIE-253269,2023-08-09,18:53:08,P-129854
